#Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install transformers pillow qdrant-client torch requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.


In [ ]:
import json

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {str(e)}")
        return None

# Usage
file_path = r"/content/drive/MyDrive/iu_xray/annotation.json"
data = read_json_file(file_path)

if data is not None:
    print("JSON data loaded successfully")
    # You can now work with the json_data
else:
    print("Failed to load JSON data")

JSON data loaded successfully


In [ ]:
def create_json(data):
    result = []
    for item in data['train']:
        id_value = item['id']
        report = item['report']
        image_0 = f"/content/drive/MyDrive/iu_xray/images/{id_value}/0.png"
        image_1 = f"/content/drive/MyDrive/iu_xray/images/{id_value}/1.png"
        result.append({
            "ID": id_value,
            "report": report,
            "image_0": image_0,
            "image_1": image_1
        })
    return result

In [ ]:
result = create_json(data)

In [ ]:
import pandas as pd
df = pd.DataFrame(result)

In [ ]:
df

,ID,report,image_0,image_1
0,CXR2384_IM-0942,The heart size and pulmonary vascularity appea...,/content/drive/MyDrive/iu_xray/images/CXR2384_...,/content/drive/MyDrive/iu_xray/images/CXR2384_...
1,CXR2926_IM-1328,Cardiac and mediastinal contours are within no...,/content/drive/MyDrive/iu_xray/images/CXR2926_...,/content/drive/MyDrive/iu_xray/images/CXR2926_...
2,CXR1451_IM-0291,Left lower lobe calcified granuloma. Heart siz...,/content/drive/MyDrive/iu_xray/images/CXR1451_...,/content/drive/MyDrive/iu_xray/images/CXR1451_...
3,CXR2887_IM-1289,The cardiomediastinal silhouette is normal in ...,/content/drive/MyDrive/iu_xray/images/CXR2887_...,/content/drive/MyDrive/iu_xray/images/CXR2887_...
4,CXR1647_IM-0424,"The lungs are clear bilaterally. Specifically,...",/content/drive/MyDrive/iu_xray/images/CXR1647_...,/content/drive/MyDrive/iu_xray/images/CXR1647_...
...,...,...,...,...
2064,CXR1600_IM-0390,The lungs are clear. There is no focal airspac...,/content/drive/MyDrive/iu_xray/images/CXR1600_...,/content/drive/MyDrive/iu_xray/images/CXR1600_...
2065,CXR2465_IM-0997,Heart size is mildly enlarged but stable.. The...,/content/drive/MyDrive/iu_xray/images/CXR2465_...,/content/drive/MyDrive/iu_xray/images/CXR2465_...
2066,CXR1595_IM-0386,The heart is large. Lung volumes are XXXX. XXX...,/content/drive/MyDrive/iu_xray/images/CXR1595_...,/content/drive/MyDrive/iu_xray/images/CXR1595_...
2067,CXR911_IM-2417,Heart size within normal limits and cardiomedi...,/content/drive/MyDrive/iu_xray/images/CXR911_I...,/content/drive/MyDrive/iu_xray/images/CXR911_I...


In [12]:
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

#Using pubmed-clip-vit-base-patch32 to generate multimodal embeddings



In [8]:
# Load the PubMed CLIP model and processor
model = CLIPModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
url= "UR_URL"

#url = "https://<your-cluster-name>.qdrant.cloud"
api_key = "API_KEY"

# Initialize Qdrant Client
qdrant = QdrantClient(url=url, api_key=api_key, prefer_grpc=False)

In [10]:
# Create a Qdrant collection
qdrant.recreate_collection(
    collection_name="chest_xray_embeddings",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE)
)

<ipython-input-10-45e91f5648b9>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [14]:
import uuid
def get_image_embedding(image_path):
    """Extract embedding for an image."""
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = model.get_image_features(inputs["pixel_values"])
    return image_features.squeeze().tolist()

def get_text_embedding(text):
    """Extract embedding for text."""
    inputs = processor(text=text, return_tensors="pt", padding=True, truncation=True,max_length=model.config.text_config.max_position_embeddings)
    with torch.no_grad():
        text_features = model.get_text_features(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    return text_features.squeeze().tolist()

def add_to_qdrant(image_path, text_summary, idx):
    """Extract embeddings and add them to Qdrant."""
    image_embedding = get_image_embedding(image_path)
    text_embedding = get_text_embedding(text_summary)
    qdrant.upsert(
        collection_name="chest_xray_embeddings",
        points=[
            {
                "id": str(idx),
                "vector": image_embedding,
                "payload": {"summary": text_summary, "image_path": image_path}
            }
        ]
    )


def generate_uuid_from_id(row_id):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, row_id))

In [ ]:
for idx, row in df.iterrows():
    point_id = generate_uuid_from_id(row["ID"])
    add_to_qdrant(row["image_1"], row["report"], point_id)

KeyboardInterrupt: 

In [ ]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00


In [ ]:
pip install LlamaCpp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.5-cp310-cp310-linux_x86_64.whl size=3471473 sha256=60ce7304d5c474daa19dc2b398e89d195d53ec68386f795278165ef074c61d85
  Stored in directory: /root/.cache/pip/wheels/7e/ae/c3/34a10eba5fc4e75df17818312b6a1ee29eb90f13a12c524714
Successfully built llama-cpp-python


In [ ]:
pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [ ]:
pip install qdrant-client


In [ ]:
from langchain import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain_community.embeddings import SentenceTransformerEmbeddings
from fastapi import FastAPI, Request, Form, Response
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi.encoders import jsonable_encoder
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
import os
import json

In [ ]:
!wget https://huggingface.co/pavel-tolstyko/ggml-model-Q4_K_M/resolve/main/ggml-model-Q4_K_M.gguf

--2024-12-12 04:17:46--  https://huggingface.co/pavel-tolstyko/ggml-model-Q4_K_M/resolve/main/ggml-model-Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.169.137.111, 3.169.137.119, 3.169.137.19, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.111|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/86/80/86806f8508c33caded0766ae00a30d63bb41e150b81819723fc8c77a5faf4c8a/ef2a6c6f4c11572102fc6faec5bbecfff9511ba08ca06361d371da41125c3076?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ggml-model-Q4_K_M.gguf%3B+filename%3D%22ggml-model-Q4_K_M.gguf%22%3B&Expires=1734236266&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDIzNjI2Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg2LzgwLzg2ODA2Zjg1MDhjMzNjYWRlZDA3NjZhZTAwYTMwZDYzYmI0MWUxNTBiODE4MTk3MjNmYzhjNzdhNWZhZjRjOGEvZWYyYTZjNmY0YzExNTcyMTAyZmM2ZmFlYzViYmVjZmZmOTUxMWJhMDhjYTA2MzYxZDM3MWRhN

In [ ]:
local_llm = "/content/ggml-model-Q4_K_M.gguf"

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path= local_llm,
    temperature=0.3,
    max_tokens=2048,
    top_p=1
)

print("LLM Initialized....")


llama_model_loader: loaded meta data with 32 key-value pairs and 201 tensors from /content/ggml-model-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = TinyLlama 1.1B Chat v1.0
llama_model_loader: - kv   3:                            general.version str              = v1.0
llama_model_loader: - kv   4:                           general.finetune str              = Chat
llama_model_loader: - kv   5:                           general.basename str              = TinyLlama
llama_model_loader: - kv   6:                         general.size_label str              = 1.1B
llama_model_loader: - kv   7:                          

LLM Initialized....



Using chat eos_token: </s>
Using chat bos_token: <s>


In [ ]:
prompt_template = """Generate an impression summary for the radiology report using the context given.
    Strictly follow the instructions below while generating the impressions.
    Instructions:
    • Impression summary should be based on the information in the context.
    • Limit the generation to {maxlen} words.
    • The impression should not mention anything about follow-up actions.
    • Impression should not contain any mentions of prior or previous studies.
    CONTEXT: {context}
    Impression summary:
    """

In [ ]:
url= 'Token'

#url = "https://<your-cluster-name>.qdrant.cloud"
api_key = "UR API KEY"
# Initialize Qdrant Client
client = QdrantClient(url=url, api_key=api_key, prefer_grpc=False)

In [ ]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context'])

In [ ]:
pip install langchain_community

In [ ]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context'])

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from sentence_transformers import SentenceTransformer
from PIL import Image
import torch
import torch.nn as nn

def get_context_from_image(image_embedding, k=1):
    """Retrieve the closest embeddings from Qdrant as context."""

    results = client.search(
        collection_name="chest_xray_embeddings",  # Replace with your collection name
        query_vector=image_embedding,             # Image embedding to search for
        limit=k,                                  # Limit the results to top k
        with_payload=True                         # Include the payload (metadata) in the results
    )

    # Combine the retrieved documents (e.g., summaries) into one context
    context = "\n".join([result.payload['summary'] for result in results])
    return context


def get_response_old(image_path):
    # Load the image and get its embedding
    image_embedding = get_image_embedding(image_path)
    # Retrieve context from Qdrant DB using the image embedding
    k=1
    context = get_context_from_image(image_embedding)

    # Create the RetrievalQA chain
    prompt = PromptTemplate(template=prompt_template, input_variables=['context'])
    #prompt = prompt_template.format(context=context)
    chain_type_kwargs = {"prompt": prompt}
    # Initialize the QA system (you still need to specify your LLM and other arguments)
    qa = RetrievalQA.from_chain_type(
        llm=llm,                          # Your language model here (e.g., GPT-3, Llama, etc.)
        chain_type="stuff",                # Use "stuff" or another chain type as needed
        retriever=None,                    # No need for a retriever here, since context is provided manually
        return_source_documents=True,      # If you want the source documents as well
        chain_type_kwargs=chain_type_kwargs,
        verbose=True
    )

    # Prepare input data for the QA model
    input_data = {"context": context}

    # Get the response
    response = qa(input_data)

    # Extract the answer and relevant source document
    answer = response['result']
    source_document = response['source_documents'][0].page_content

    # Print the results
    print(f"Answer: {answer}")
    print(f"Source Document: {source_document}")


# Function to generate response
def get_response(image_path, k=1):
    # Load the image and get its embedding
    image_embedding = get_image_embedding(image_path)

    # Retrieve context from Qdrant DB using the image embedding

    context = get_context_from_image(image_embedding, k=k)

    # Define the custom prompt template
    prompt_template = """Generate an impression summary for the radiology report using the context given.
    Strictly follow the instructions below while generating the impressions.
    Instructions:
    • Impression summary should be based on the information in the context.
    • Limit the generation to {maxlen} words.
    • The impression should not mention anything about follow-up actions.
    • Impression should not contain any mentions of prior or previous studies.
    CONTEXT: {context}
    Impression summary:
    """

    # Format the prompt with the retrieved context
    prompt = prompt_template.format(context=context,maxlen=300)


    response = llm(prompt)

    return response



#Get the report corresponding to the image embeddings we found via semantic Search

In [ ]:
input_image_embedding = get_image_embedding('/content/1.png')
context = get_context_from_image(input_image_embedding,1)
context

'The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.'

In [ ]:
get_response('/content/1.png')

Llama.generate: 445 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   34446.73 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    66 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10629.93 ms /    67 tokens


'\n    <assistant>\n    \n    The radiology report states that the heart is normal in size and the mediastinum is unremarkable. However, the passage does not provide any information on whether there are any abnormalities or findings in the lungs. Therefore, it is recommended to exclude this'

In [ ]:
\n    <assistant>\n    \n    The radiology report states that the heart is normal in size and the mediastinum is unremarkable. However, the passage does not provide any information on whether there are any abnormalities or findings in the lungs. Therefore, it is recommended to exclude this

#our UI

In [ ]:
pip install gradio


In [ ]:
import os
import shutil
import gradio as gr
from PIL import Image as PILImage

# Define the folder where the images are uploaded
upload_folder = '/content/upload_image'

# Function to clean the folder
def clean_folder():
    if os.path.exists(upload_folder):
        shutil.rmtree(upload_folder)  # Remove the folder and its contents
    os.makedirs(upload_folder)  # Recreate the empty folder

# Function to process the uploaded image
def process_image(image):
    # Clean the folder before uploading the new image
    clean_folder()

    # Save the uploaded image to the folder
    image_path = os.path.join(upload_folder, 'uploaded_image.png')
    image.save(image_path)

    # Display the uploaded image
    img = PILImage.open(image_path)

    context = get_response(image_path,1)
    #print(context)

    # Return the image and the context
    return img, context

# Create the Gradio interface
iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),  # Upload image widget (PIL type)
    outputs=[gr.Textbox()]  # Display image and context
)

# Launch the Gradio app
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0484e95ae960366219.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import os

# Change directory to where your zip file is located
os.chdir('/content/drive')  # Adjust this path if necessary


In [16]:
df=pd.read_csv('/content/df_val.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/df_val.csv'

In [36]:
from google.colab import drive
import cv2
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Define the file path
file_path = '/content/drive/MyDrive/destination_folder/Downloads_val/CXR1141_IM-0096/CXR1141_IM-0096_image_1.jpg'
#print(get_response(file_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Integrating all RAG generated responses for our sample of 50 images

In [22]:
import pandas as pd

def update_responses(df):

    def process_id(id):
        file_path = f'/content/drive/MyDrive/destination_folder/Downloads_val/{id}/{id}_image_1.jpg'
        return get_response(file_path)

    df['response'] = df['ID'].apply(process_id)
    return df

In [24]:
df = update_responses(df)

NameError: name 'df' is not defined

In [ ]:
df.head(2)

,ID,report,image_0,image_1,response
0,CXR2384_IM-0942,The heart size and pulmonary vascularity appea...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Heart size and pulmonary vascularity appear...
1,CXR2926_IM-1328,Cardiac and mediastinal contours are within no...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,"1. Cardiac contours are within normal limits, ..."


In [ ]:
first_row_response = df_copy.loc[34, [ 'report','cleaned_response']].to_dict()


In [ ]:
first_row_response

{'report': 'The heart is normal in size. The mediastinum is unremarkable. The chest XXXX is in satisfactory position. There is no pneumothorax. The lungs are clear.',
 'cleaned_response': '1. The heart is normal in size, as per the radiology report.\n    2. The mediastinum is unremarkable, as per the radiology report.\n    3. The chest XXXX is in satisfactory position, as per the radiology report.\n    4. There is no pneumothorax, as per the radiology report.\n    5. The lungs are clear, as per the radiology report.\n    Conclusion: Based on the radiology report, there is no evidence of pneumothorax or other lung abnormalities.'}

In [ ]:
df_copy=df.copy()

In [ ]:
df_copy.head(3)

,ID,report,image_0,image_1,response
0,CXR2384_IM-0942,The heart size and pulmonary vascularity appea...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Heart size and pulmonary vascularity appear...
1,CXR2926_IM-1328,Cardiac and mediastinal contours are within no...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,"1. Cardiac contours are within normal limits, ..."
2,CXR1451_IM-0291,Left lower lobe calcified granuloma. Heart siz...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Radiographic examination:\n A calcifie...


In [ ]:
import pandas as pd
import re

#Cleaned responses for comparison

In [ ]:
pattern = re.compile(r'^\s*•.*$', re.MULTILINE)

def clean_response(response):
    return re.sub(pattern, '', response).strip()

# Apply the function to clean the responses
df_copy['cleaned_response'] = df_copy['response'].apply(clean_response)

In [ ]:
df_copy

,ID,report,image_0,image_1,response,cleaned_response
0,CXR2384_IM-0942,The heart size and pulmonary vascularity appea...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Heart size and pulmonary vascularity appear...,1. Heart size and pulmonary vascularity appear...
1,CXR2926_IM-1328,Cardiac and mediastinal contours are within no...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,"1. Cardiac contours are within normal limits, ...","1. Cardiac contours are within normal limits, ..."
2,CXR1451_IM-0291,Left lower lobe calcified granuloma. Heart siz...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Radiographic examination:\n A calcifie...,1. Radiographic examination:\n A calcifie...
3,CXR2887_IM-1289,The cardiomediastinal silhouette is normal in ...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Cardiomediastinal silhouette: Normal in siz...,1. Cardiomediastinal silhouette: Normal in siz...
4,CXR1647_IM-0424,"The lungs are clear bilaterally. Specifically,...",C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,"1. The lungs are clear bilaterally, with no ev...","1. The lungs are clear bilaterally, with no ev..."
5,CXR2434_IM-0976,The heart is normal in size. The mediastinum i...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. The heart is normal in size\n 2. The med...,1. The heart is normal in size\n 2. The med...
6,CXR3655_IM-1817,Well circumscribed 11 mm right upper lobe nodu...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Well circumscribed 11 mm right upper lobe n...,1. Well circumscribed 11 mm right upper lobe n...
7,CXR3299_IM-1575,The cardiac silhouette and mediastinal contour...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. The cardiac silhouette and mediastinal cont...,1. The cardiac silhouette and mediastinal cont...
8,CXR1419_IM-0267,Cardiac silhouette and pulmonary vascularity a...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,1. Cardiac silhouette:\n A normal cardi...,1. Cardiac silhouette:\n A normal cardi...
9,CXR2867_IM-1274,The trachea is midline. The cardiomediastinal ...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,"1. The trachea is midline, with no change from...","1. The trachea is midline, with no change from..."


#PROMPT WE USED FOR LLM as a JUDGE

In [ ]:
prompt_template = """
f"""
    Question: {question}
    Ground Truth: {ground_truth}
    LLM Only Response: {llm_only_response}
    RAG Response: {rag_response}

    Compare the LLM only response and the RAG response to the ground truth respectively on the following criteria:
    - Relevance: Does the response directly address the query?
    - Accuracy: Is the information provided correct?
        - Conciseness: Is the response free from unnecessary information?

    Thereafter, provide an overall score out of 10 for LLM Only Response and for the RAG Response respectively. Do not change the format of the output.

    ### Summary
    - LLM Only Response:

    - RAG Response:

    ### LLM Only Overall Score:
    ### RAG Overall Score:
    """


(We used AWS bedrock through which we called Calude 3.5 Sonnet)

In [ ]:
df.to_csv('/content/drive/My Drive/destination_folder/final.csv', index=False)

In [ ]:
# Define base path for all folders
base_path = '/content/drive/MyDrive/Downloads_val/destination_folder/'



# Print full paths
for folder in id_list:
    full_path = os.path.join(base_path, folder)
    print(full_path)

/content/drive/MyDrive/Downloads_val/destination_folder/CXR2279_IM-0865
/content/drive/MyDrive/Downloads_val/destination_folder/CXR2056_IM-0694-1001
/content/drive/MyDrive/Downloads_val/destination_folder/CXR2379_IM-0939
/content/drive/MyDrive/Downloads_val/destination_folder/CXR3586_IM-1764
/content/drive/MyDrive/Downloads_val/destination_folder/CXR3149_IM-1480
/content/drive/MyDrive/Downloads_val/destination_folder/CXR3804_IM-1914
/content/drive/MyDrive/Downloads_val/destination_folder/CXR2316_IM-0889
/content/drive/MyDrive/Downloads_val/destination_folder/CXR3717_IM-1856
/content/drive/MyDrive/Downloads_val/destination_folder/CXR3289_IM-1570
/content/drive/MyDrive/Downloads_val/destination_folder/CXR1141_IM-0096
/content/drive/MyDrive/Downloads_val/destination_folder/CXR357_IM-1753
/content/drive/MyDrive/Downloads_val/destination_folder/CXR1553_IM-0360
/content/drive/MyDrive/Downloads_val/destination_folder/CXR667_IM-2241
/content/drive/MyDrive/Downloads_val/destination_folder/CXR27

In [31]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info


model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [37]:
df_val=pd.read_csv(r"/content/sample_data/df_val.csv")

In [45]:
df_val.head(3)

,ID,report,image_0,image_1
0,CXR2384_IM-0942,The heart size and pulmonary vascularity appea...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...
1,CXR2926_IM-1328,Cardiac and mediastinal contours are within no...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...
2,CXR1451_IM-0291,Left lower lobe calcified granuloma. Heart siz...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...,C:\Users\njain\Downloads\iu_xray\iu_xray\image...


In [56]:
folder_path = f'/content/drive/MyDrive/destination_folder/Downloads_val/{id}/{id}_image_1.jpg'

#THIS IS THE LLM GENERATED RESPONSE SECTION

In [63]:
pip install google-generativeai

In [65]:
pip install google-generativeai


In [2]:
import google.generativeai as genai
from IPython.display import Markdown

genai.configure(api_key='Your_key')

In [70]:
model = genai.GenerativeModel('gemini-pro-vision')

In [6]:
import os
import base64
import pandas as pd
from google.generativeai import GenerativeModel
from IPython.display import Markdown
from PIL import Image

# Define the base folder path
folder_path = '/content/drive/MyDrive/destination_folder/Downloads_val/'

# Initialize the Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Initialize an empty list to store the results
results = []

def generate_image_caption(image_path):
    try:
        # Open and validate the image
        with Image.open(image_path) as img:
            img_format = img.format
            print(f"Processing image: {image_path}, Format: {img_format}, Size: {img.size}")

            # Ensure the image is in a valid format
            if img_format not in ["JPEG", "PNG"]:
                raise ValueError("Unsupported image format. Please use JPEG or PNG.")

            # Convert the image to Base64
            with open(image_path, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode("utf-8")

        # Create a prompt
        prompt = "Explain this image"

        # Generate caption using Gemini
        response = model.generate_content(
            [
                {
                    "mime_type": "image/jpeg",  # Update if the API accepts PNG
                    "data": image_data,
                },
                prompt,
            ]
        )

        return response.text
    except Exception as e:
        raise ValueError(f"Error generating caption for image: {image_path}. {str(e)}")

# Process images in subfolders
for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(subfolder_path, file)
                try:
                    caption = generate_image_caption(image_path)
                    image_id = os.path.splitext(file)[0]  # Extract image ID from filename

                    # Add the result to the list
                    results.append({"Image ID": image_id, "Caption": caption})

                    print(f"Caption: {caption}\n")
                    print(Markdown(">" + caption))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}\n")




Processing image: /content/drive/MyDrive/destination_folder/Downloads_val/CXR1135_IM-0091/CXR1135_IM-0091_image_0.jpg, Format: PNG, Size: (512, 624)


KeyboardInterrupt: 

#WE OBTAINED THE LLM RESPONSE (without RAG using below)

In [9]:
# Process images in subfolders
for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(subfolder_path, file)
                try:
                    print(image_path)
                    image_id = os.path.splitext(file)[0]  # Extract image ID from filename

                    # Add the result to the list
                    results.append({"Image ID": image_id, "Caption": caption})

                    print(f"Caption: {caption}\n")
                    print(Markdown(">" + caption))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}\n")

Streaming output truncated to the last 5000 lines.
* **Cystic Fibrosis:**  While less common, cystic fibrosis can cause lung changes and hyperinflation.
* **Alpha-1 Antitrypsin Deficiency:** This genetic disorder can lead to emphysema-like changes in the lungs.


**Important Note:** This interpretation is based solely on a single X-ray image. A definitive diagnosis requires a complete medical history, physical examination, and potentially further investigations such as pulmonary function tests (PFTs).  A radiologist's formal reading of the image would also be crucial.  This information should not be used for self-diagnosis or treatment.  If you have concerns about your respiratory health, please consult a medical professional.


<IPython.core.display.Markdown object>
/content/drive/MyDrive/destination_folder/Downloads_val/CXR1167_IM-0112/CXR1167_IM-0112_image_0.jpg
Caption: This is a posterior-anterior (PA) chest X-ray.  Here's a breakdown of what can be seen and some possible interpre

In [8]:
caption

"This is a posterior-anterior (PA) chest X-ray.  Here's a breakdown of what can be seen and some possible interpretations:\n\n* **Overall Appearance:** The lungs appear overexpanded, with flattening of the diaphragm. This suggests hyperinflation, often seen in obstructive lung diseases like emphysema or chronic bronchitis (components of COPD).\n\n* **Lung Fields:**  There is increased lucency (darkness) throughout both lung fields, which further supports the idea of hyperinflation.  There are also some subtle linear markings that could represent fibrosis or other changes, but it's hard to be definitive without a higher resolution image and clinical context.\n\n* **Diaphragm:** As mentioned, the diaphragm appears flattened, lower than typical. This is another sign of hyperinflation.\n\n* **Heart:** The heart size appears within normal limits on this view.\n\n* **Bony Structures:** The ribs appear somewhat thin, possibly again related to chronic lung disease.  The clavicles and other bon

In [12]:
results

[{'Image ID': 'CXR1135_IM-0091_image_0',
  'Caption': "This is a posterior-anterior (PA) chest X-ray.  Here's a breakdown of what can be seen and some possible interpretations:\n\n* **Overall Appearance:** The lungs appear overexpanded, with flattening of the diaphragm. This suggests hyperinflation, often seen in obstructive lung diseases like emphysema or chronic bronchitis (components of COPD).\n\n* **Lung Fields:**  There is increased lucency (darkness) throughout both lung fields, which further supports the idea of hyperinflation.  There are also some subtle linear markings that could represent fibrosis or other changes, but it's hard to be definitive without a higher resolution image and clinical context.\n\n* **Diaphragm:** As mentioned, the diaphragm appears flattened, lower than typical. This is another sign of hyperinflation.\n\n* **Heart:** The heart size appears within normal limits on this view.\n\n* **Bony Structures:** The ribs appear somewhat thin, possibly again related

In [13]:
df_r = pd.DataFrame(results)

In [15]:
df_r['Image ID'].nunique()

200

In [16]:
df_r = df_r.drop_duplicates(subset='Image ID')

In [17]:
df_r

,Image ID,Caption
0,CXR1135_IM-0091_image_0,This is a posterior-anterior (PA) chest X-ray....
1,CXR1135_IM-0091_image_1,This is a posterior-anterior (PA) chest X-ray....
2,CXR1141_IM-0096_image_0,This is a posterior-anterior (PA) chest X-ray....
3,CXR1141_IM-0096_image_1,This is a posterior-anterior (PA) chest X-ray....
4,CXR1145_IM-0097_image_0,This is a posterior-anterior (PA) chest X-ray....
...,...,...
195,CXR917_IM-2419_image_1,This is a posterior-anterior (PA) chest X-ray....
196,CXR930_IM-2429_image_0,This is a posterior-anterior (PA) chest X-ray....
197,CXR930_IM-2429_image_1,This is a posterior-anterior (PA) chest X-ray....
198,CXR951_IM-2447_image_0,This is a posterior-anterior (PA) chest X-ray....


In [21]:
df_filtered.to_csv('/content/drive/My Drive/destination_folder/df_filtered.csv', index=False)

In [19]:
df_filtered = df_r[~df_r['Image ID'].str.endswith('_image_0')]

,Image ID,Caption
1,CXR1135_IM-0091_image_1,This is a posterior-anterior (PA) chest X-ray....
3,CXR1141_IM-0096_image_1,This is a posterior-anterior (PA) chest X-ray....
5,CXR1145_IM-0097_image_1,This is a posterior-anterior (PA) chest X-ray....
7,CXR1151_IM-0102_image_1,This is a posterior-anterior (PA) chest X-ray....
9,CXR1167_IM-0112_image_1,This is a posterior-anterior (PA) chest X-ray....
...,...,...
191,CXR726_IM-2286_image_1,This is a posterior-anterior (PA) chest X-ray....
193,CXR880_IM-2395_image_1,This is a posterior-anterior (PA) chest X-ray....
195,CXR917_IM-2419_image_1,This is a posterior-anterior (PA) chest X-ray....
197,CXR930_IM-2429_image_1,This is a posterior-anterior (PA) chest X-ray....
